# Tito Analytics
---

### initialize script
run this first always

In [ ]:
%%capture
from IPython.core.magic import register_line_magic

@register_line_magic
def pip(args):
    """Use pip from the current kernel"""
    from pip import main
    main(args.split())
    
%pip install pandas
%pip install geopy
%pip install openpyxl

from geopy.geocoders import Nominatim
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 4000)
import datetime
geolocator = Nominatim(user_agent="TitoApp")

class analytics:
    FILES = {        
        "EXCEL_FILE": "TitoDataProd.xlsx",
        "USERS_MOBILE": "Users Mobile",
        "USERS_SETTINGS": "mobile_settings.csv",
        "POIS": "pois.csv",
        "POI_PICTURES": "poi_pictures.csv",
        "LIKES_TAB": "likes.csv",
        "COMMENTS_TAB" : "comments.csv"
    }
    
    fields = {
        "total_current_users": None,
        "total_MAU_users": None,
        "pins_generated": None,
        "objects_uploaded": None,
        "pins_from_broadcasts": None,
        "object_threes": None,
        "notifications_on_total": None,
        "notifications_on_MAU": None,
        "location_never": None,
        "precise_location_off": None,
        "gallery_permissions_off": None,
        "camera_permissions_off": None,
        "one_time_users": None,
        "new_weekly_users": None,
        "users_nyc": None,
        "users_nyc_mau": None,
    }
    
    date_of_sheet = None
    MAU_date = None
    WAU_date = None
    df = None
    users_mobile_df = None
    users_settings_df = None
    pois_df = None
    pictures_df = None
    current_df = None
    likes_df = None
    comments_df = None
    
    weights = {'post_weight':1, 'comment_weight':1/3, 'like_weight':1/9}
    
    def __init__(self, date):
        self.init_dates(date)
        self.open_files()
        self.merge_dfs()
        self.format_all_dates()
        self.make_current_users_df()
        self.fill_fields()
        
    def init_dates(self, date):
        date_time_str = f'{date} 23:59:59'
        self.date_of_sheet = datetime.datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')
        self.MAU_date = self.date_of_sheet - datetime.timedelta(28)
        self.WAU_date = self.date_of_sheet - datetime.timedelta(7)
        
    def open_sheet(self, sheet_name):
        return pd.read_excel(self.FILES["EXCEL_FILE"], engine='openpyxl', sheet_name=sheet_name)
        
    def open_files(self):
        self.users_mobile_df = self.open_sheet(self.FILES["USERS_MOBILE"])
        self.users_settings_df = self.open_sheet(self.FILES["USERS_SETTINGS"])
        self.pois_df = self.open_sheet(self.FILES["POIS"])
        self.pictures_df = self.open_sheet(self.FILES["POI_PICTURES"])
        self.likes_df = self.open_sheet(self.FILES["LIKES_TAB"])
        self.comments_df = self.open_sheet(self.FILES["COMMENTS_TAB"])

        
    def merge_dfs(self):
        self.df = pd.merge(self.users_mobile_df, self.users_settings_df.drop(['id', 'created_at', 'updated_at']
                            , axis=1),left_on='id', right_on='user_id', how='left').drop('user_id', axis=1)
      
    def format_date(self, data_frame, column):
        data_frame[column] = pd.to_datetime(data_frame[column], errors='coerce')
    
    def format_all_dates(self):
        self.format_date(self.df, 'last_activity_timestamp')
        self.format_date(self.df, 'created_at')
        self.format_date(self.pois_df, 'created_at')
        self.format_date(self.pictures_df, 'created_at')
        self.format_date(self.likes_df, 'created_at')
        self.format_date(self.comments_df, 'created_at')
        
    def make_current_users_df(self):
        self.df_current = self.df[self.df['mobile_version'] \
                                  .str.match("^[1-9][0-9]*\.\d+\.\d+ \(\d+\)|0.1.0 \(65\)") == True]
        
    def fill_fields(self):
        self.fields['total_current_users'] = self.df_current.shape[0]
        
        t = self.pois_df.apply(lambda x: True if x['created_at'] > self.WAU_date else False, axis=1)
        self.fields['pins_generated'] = len(t[t].index)

        t = self.pois_df.apply(lambda x: True if x['created_at'] > self.WAU_date 
                               and x['accredited_user_id'] > 0 else False, axis=1)
        self.fields['pins_from_broadcasts'] = len(t[t].index)

        t = self.pictures_df.apply(lambda x: True if x['created_at'] > self.WAU_date else False, axis=1)
        self.fields['objects_uploaded'] = len(t[t].index)

        t = self.pictures_df.apply(lambda x: True if x['created_at'] > self.WAU_date 
                                   and x['great_count'] > 0 else False, axis=1)
        self.fields['object_threes'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['notification_permissions'] == 'allowed' else False, axis=1)
        self.fields['notifications_on_total'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['last_activity_timestamp'] > self.MAU_date else False, axis=1)
        self.fields['total_MAU_users'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['notification_permissions'] == 'allowed' 
                                  and x['last_activity_timestamp'] > self.MAU_date else False, axis=1)
        self.fields['notifications_on_MAU'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['location_permissions'] == 'never' else False, axis=1)
        self.fields['location_never'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['precise_location'] == 'OFF' else False, axis=1)
        self.fields['precise_location_off'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['gallery_permissions'] == 'never' 
                                  or x['gallery_permissions'] == 'not-allowed' 
                                  or x['gallery_permissions'] == 'Limited' else False, axis=1)
        self.fields['gallery_permissions_off'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['camera_permissions'] == 'never' 
                                  or x['camera_permissions'] == 'not-allowed' else False, axis=1)
        self.fields['camera_permissions_off'] = len(t[t].index)

        t = self.df_current.apply(lambda x: True if pd.isnull(x['notification_permissions']) 
                                  and pd.isnull(x['location_permissions']) 
                                  and pd.isnull(x['gallery_permissions']) and pd.isnull(x['camera_permissions']) 
                                  and pd.isnull(x['microphone_permissions']) else False, axis=1)
        self.fields['one_time_users'] = len(t[t].index)
        
        t = self.df.apply(lambda x: True if x['created_at'] > self.WAU_date else False, axis=1)
        self.fields['new_weekly_users'] = len(t[t].index)
        
    def print_analytics(self):
        total_current_users = self.fields['total_current_users']
        total_MAU_users = self.fields['total_MAU_users']
        pins_generated = self.fields['pins_generated']
        objects_uploaded = self.fields['objects_uploaded']
        pins_from_broadcasts = self.fields['pins_from_broadcasts']
        object_threes = self.fields['object_threes']
        notifications_on_total = self.fields['notifications_on_total']
        notifications_on_MAU = self.fields['notifications_on_MAU']
        location_never = self.fields['location_never']
        precise_location_off = self.fields['precise_location_off']
        gallery_permissions_off = self.fields['gallery_permissions_off']
        camera_permissions_off = self.fields['camera_permissions_off']
        one_time_users = self.fields['one_time_users']
        new_weekly_users = self.fields['new_weekly_users']

        
        print(f"# Total Users: {total_current_users}")
        print(f"# Total MAU Users: {total_MAU_users}")
        print(f"# PINs Generated (All): {pins_generated}")
        print(f"# Objects Uploaded: {objects_uploaded}")
        print(f"# PINs from Broadcasts: {pins_from_broadcasts}")
        print(f"# Objects 3s (make the feed): {object_threes}")
        print(f"# Notifications On: {notifications_on_total}")
        print(f"% Notifications On: {round(notifications_on_total/total_current_users*100, 1)}%")
        print(f"# Notifications On (MAU): {notifications_on_MAU}")
        print(f"% Notifications On (MAU): {round(notifications_on_MAU/total_MAU_users*100,1)}%")
        print(f"# Location: Never: {location_never}")
        print(f"% Location: Never: {round(location_never/total_current_users*100, 1)}%")
        print(f"# Precise Location Off: {precise_location_off}")
        print(f"% Precise Location Off: {round(precise_location_off/total_current_users*100, 1)}%")
        print(f"# Photos: Limited, Never or Not Allowed: {gallery_permissions_off}")
        print(f"% Photos: Limited, Never or Not Allowed: {round(gallery_permissions_off/total_current_users*100, 1)}%")
        print(f"# Camera: Never or Not Allowed: {camera_permissions_off}")
        print(f"% Camera: Never or Not Allowed: {round(camera_permissions_off/total_current_users*100, 1)}%")
        print(f"# Mobile Settings Unavailable (one-time users): {one_time_users}")
        print(f"% Mobile Settings Unavailable (one-time users): {round(one_time_users/total_current_users*100, 1)}%")
        print(f"# New Weekly Users: {new_weekly_users}")

        
    def get_location_data(self):
        self.df_current['postcode'] = self.df_current.apply(lambda x: self.get_zip(x), axis=1)

        t = self.df_current.apply(lambda x: True if x['postcode'] is not None else False, axis=1)
        users_nyc = len(t[t].index)

        t = self.df_current.apply(lambda x: True if x['postcode'] is not None 
                                  and x['last_activity_timestamp'] > self.MAU_date else False, axis=1)
        users_nyc_mau = len(t[t].index)

        print(f"# Total NYC users: {users_nyc}")
        print(f"# MAU NYC users: {users_nyc_mau}")
        
    def print_zip(self):
        print("Users per zipcode:")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
            print(self.df_current['postcode'].value_counts())
        
        
    def get_zip(self, x):
        if x['last_lat'] != 0 and not pd.isnull(x['last_lat']) and x['last_lng'] != 0 and not pd.isnull(x['last_lng']):
            last_location = f"{x['last_lat']}, {x['last_lng']}"
        else:
            last_location = f"{x['lat']}, {x['lng']}"
        try:
            loc = geolocator.reverse(last_location)
            if loc.raw['address']['city'] == 'New York':
                return loc.raw['address']['postcode']
        except:
            return None
        
    def get_city(self, x):
        if x['last_lat'] != 0 and not pd.isnull(x['last_lat']) and x['last_lng'] != 0 and not pd.isnull(x['last_lng']):
            last_location = f"{x['last_lat']}, {x['last_lng']}"
        else:
            last_location = f"{x['lat']}, {x['lng']}"
        try:
            loc = geolocator.reverse(last_location)
            return loc.raw['address']['city']

        except:
            return None
        
    def get_rem(self):
        self.df['city'] = self.df.apply(lambda x: self.get_city(x), axis=1)
        self.df['days_since_activity'] = self.df.dropna(subset=['last_activity_timestamp']).apply(lambda row: round((self.date_of_sheet - row['last_activity_timestamp']) / datetime.timedelta(days=1)), axis=1)
        self.df[['id', 'username', 'email', 'days_since_activity', 'notification_permissions', 'location_permissions', 'city']].set_index('id').to_excel("rem_data.xlsx")  
    
    def score_calculator(self, row):
        return round(row['posts_count'] * self.weights['post_weight'] + row['comments_count'] * self.weights['comment_weight'] + row['likes_count'] * self.weights['like_weight'], 1)
    
    def get_user_stats(self):
        self.generate_time_stats(name="All Time Ratings")
        self.generate_time_stats(date=self.MAU_date, name="Monthly Ratings")
        self.generate_time_stats(date=self.WAU_date, name="Weekly Ratings")
    
    def generate_time_stats(self, name, date=None):
        if date:
            pictures_df = self.pictures_df[self.pictures_df['created_at'] > date]
            likes_df = self.likes_df[self.likes_df['created_at'] > date]
            comments_df = self.comments_df[self.comments_df['created_at'] > date]
        else:
            pictures_df = self.pictures_df
            likes_df = self.likes_df
            comments_df = self.comments_df
            
        picture_counts = pictures_df['user_id'].value_counts().reset_index().rename(columns={'index': 'user_id', 'user_id':'posts_count',0: 'count'})
        like_counts = likes_df['user_id'].value_counts().reset_index().rename(columns={'index': 'user_id', 'user_id':'likes_count',0: 'count'})
        comment_counts = comments_df['user_id'].value_counts().reset_index().rename(columns={'index': 'user_id', 'user_id':'comments_count',0: 'count'})

        df_stats = pd.merge(self.users_mobile_df, picture_counts, left_on='id', right_on='user_id', how='left').drop('user_id', axis=1)
        df_stats = pd.merge(df_stats, like_counts, left_on='id', right_on='user_id', how='left').drop('user_id', axis=1)
        df_stats = pd.merge(df_stats, comment_counts, left_on='id', right_on='user_id', how='left').drop('user_id', axis=1)

        df_stats['posts_count'] = df_stats['posts_count'].fillna(0)
        df_stats['likes_count'] = df_stats['likes_count'].fillna(0)
        df_stats['comments_count'] = df_stats['comments_count'].fillna(0)

        df_stats['score'] = df_stats.apply(lambda row: self.score_calculator(row), axis=1)
        df_stats = df_stats[['id', 'email', 'username', 'first_name', 'last_name','posts_count','likes_count','comments_count','score']].sort_values(by=["score"], ascending=False)
        df_stats = df_stats.set_index('id')
        
        try:
            with pd.ExcelWriter('user_stats.xlsx', mode='a') as writer:
                df_stats.to_excel(writer,sheet_name=name)
        except FileNotFoundError:
            with pd.ExcelWriter('user_stats.xlsx') as writer:
                df_stats.to_excel(writer,sheet_name=name)
        

### initialize analytics with date of sheet
always run this cell

In [ ]:
a = analytics("28/01/21")

---
---
---

# Analytics

### print analytics

In [ ]:
a.print_analytics()

---

# Location

### get location data
takes some time to retrieve data

In [ ]:
a.get_location_data()

### get zip data
run this only after running above cell (get_location_data())

In [ ]:
a.print_zip()

---

# Generate Sheets

### get REM sheet
Takes some time to retrieve location data.<br />
Will output an excel sheet named "rem_data.xlsx" in the outer directory.

In [ ]:
a.get_rem()

### get user statistics sheet
Will output an excel sheet named "user_stats.xlsx" in the outer directory. <br />
The way the score is calculated is: __score = post_count * post_weight + comment_count * comment_weight + like_count * like_weight__


In [ ]:
a.weights = {
    'post_weight': 1
    , 
    'comment_weight': 1/3
    , 
    'like_weight': 1/9
}
a.get_user_stats()